In [1]:
%pip install selenium

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: python-apt 2.4.0-ubuntu3-zorin1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-apt or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sys

sys.path.insert(0, "/usr/lib/chromium-browser/chromedriver")
from selenium import webdriver
import chromedriver_autoinstaller

import json
from tqdm.auto import tqdm
import requests
import os
from selenium.webdriver.common.by import By
import json
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from dotenv import load_dotenv
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from tqdm.auto import tqdm

In [38]:
class VerifiedChannelScraping:

    def __init__(self):
        with open("medical_fields.json") as f:
            self.medical_fields = json.load(f)

    def get_driver(self):
        capabilities = DesiredCapabilities.CHROME
        capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
        chrome_options = webdriver.ChromeOptions()
        # chrome_options.add_argument("--headless")  # this is must
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chromedriver_autoinstaller.install()
        driver = webdriver.Chrome(options=chrome_options)
        return driver

    def get_channels_from_source(self, source):
        channels = []
        lines = source.split("\n")
        for line in lines:
            idx1 = line.find('href="/@')
            if idx1 > -1:
                idx1 += 7
                idx2 = line[idx1:].find('"')
                channels.append(line[idx1 : idx1 + idx2])
        channels = list(set(channels))

        return channels

    def get_channel_candidates(self, driver):
        channels = []
        actions = ActionChains(driver)

        for search in tqdm(self.medical_fields):
            search += " Behandlungsmöglichkeiten"
            driver.get(f"https://www.youtube.com/results?search_query={search}")
            for i in tqdm(range(50)):
                time.sleep(1)
                actions.send_keys(Keys.END).perform()
            new_channels = self.get_channels_from_source(driver.page_source)
            channels.extend(new_channels)
            channels = list(set(channels))

        return channels

    def _get_video_from_channel(self, source):
        idx1 = source.find("/watch?")
        if idx1 > -1:
            idx1 += 9
            idx2 = source[idx1:].find('"')
            return source[idx1 : idx1 + idx2]

    def _is_verified_from_video(self, source):
        return (
            source.find(
                "M17 13h-4v4h-2v-4H7v-2h4V7h2v4h4v2zm-5-9.9 7 3.21v4.84c0 1.3-.25 2.6-.75 3.86-.15.37-.33.76-.55 1.17-.15.27-.31.54-.48.81-1.31 2.01-3.16 3.42-5.22 3.98-2.06-.56-3.91-1.97-5.23-3.98-.17-.27-.33-.54-.48-.81-.22-.41-.4-.79-.55-1.17-.49-1.26-.74-2.56-.74-3.86V6.31l7-3.21M12 2 4 5.67v5.49c0 1.47.3 2.9.81 4.22.17.44.37.86.6 1.28.16.3.34.6.52.88 1.42 2.17 3.52 3.82 5.95 4.44L12 22l.12-.03c2.43-.61 4.53-2.26 5.95-4.43.19-.29.36-.58.52-.88.22-.41.43-.84.6-1.28.51-1.33.81-2.76.81-4.23V5.67L12 2z"
            )
            > -1
        )

    def get_verified_channels(self, channel_candidates, driver):
        verified_channels = []

        for channel in tqdm(channel_candidates):
            driver.get(f"https://www.youtube.com/{channel}/videos")
            video = self._get_video_from_channel(driver.page_source)
            driver.get(f"https://www.youtube.com/watch?v={video}")
            time.sleep(8)
            verified = self._is_verified_from_video(driver.page_source)
            print(verified)
            if verified:
                verified_channels.append(channel)

        return verified_channels


scraping = VerifiedChannelScraping()

In [24]:
driver = scraping.get_driver()

In [ ]:
candidates = scraping.get_channel_candidates(driver)

In [40]:
len(candidates)

8378

In [39]:
verified = scraping.get_verified_channels(candidates, driver)

  0%|          | 0/8378 [00:00<?, ?it/s]

False
False
False
False
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fa

MaxRetryError: HTTPConnectionPool(host='localhost', port=38227): Max retries exceeded with url: /session/df4409257aa4f60ffbbda6a35a4e98ca/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x76ba39e39270>: Failed to establish a new connection: [Errno 111] Connection refused'))